# Plot Exclusions Validation

This notebook validates that plot exclusions are working correctly for different regions and categories.

## Overview

- Test 1b SR exclusions (jet3 + leptons)
- Test 2b SR exclusions (leptons only)
- Test Top CR exclusions (z_mass, z_pt)
- Test W CR exclusions (z_mass, z_pt)
- Test Z CR inclusions (z_mass, z_pt should be included)
- Test custom exclusions from config


In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from darkbottomline.plotting import PlotManager
import yaml


In [ ]:
# Initialize plot manager
plot_manager = PlotManager()

# Test regions
test_regions = [
    "1b:SR",
    "2b:SR",
    "1b:CR_Wlnu_mu",
    "1b:CR_Wlnu_el",
    "2b:CR_Top_mu",
    "2b:CR_Top_el",
    "1b:CR_Zll_mu",
    "1b:CR_Zll_el",
    "2b:CR_Zll_mu",
    "2b:CR_Zll_el",
]

print("Testing plot exclusions for different regions:\n")
for region in test_regions:
    excluded = plot_manager._get_excluded_variables_for_region(region)
    print(f"{region}:")
    print(f"  Excluded variables ({len(excluded)}): {', '.join(excluded[:10])}")
    if len(excluded) > 10:
        print(f"  ... and {len(excluded) - 10} more")
    print()


In [ ]:
# Test specific exclusions
print("=== 1b SR Exclusions ===")
region = "1b:SR"
excluded = plot_manager._get_excluded_variables_for_region(region)
assert "jet3_pt" in excluded, "jet3_pt should be excluded from 1b SR"
assert "lep1_pt" in excluded, "lep1_pt should be excluded from 1b SR"
assert "n_muons" in excluded, "n_muons should be excluded from 1b SR"
print(f"✓ {region} correctly excludes jet3 and lepton variables\n")

print("=== 2b SR Exclusions ===")
region = "2b:SR"
excluded = plot_manager._get_excluded_variables_for_region(region)
assert "lep1_pt" in excluded, "lep1_pt should be excluded from 2b SR"
assert "jet3_pt" not in excluded, "jet3_pt should NOT be excluded from 2b SR"
print(f"✓ {region} correctly excludes lepton variables but includes jet3\n")

print("=== Top CR Exclusions ===")
region = "2b:CR_Top_mu"
excluded = plot_manager._get_excluded_variables_for_region(region)
assert "z_mass" in excluded, "z_mass should be excluded from Top CR"
assert "z_pt" in excluded, "z_pt should be excluded from Top CR"
print(f"✓ {region} correctly excludes z_mass and z_pt\n")

print("=== W CR Exclusions ===")
region = "1b:CR_Wlnu_mu"
excluded = plot_manager._get_excluded_variables_for_region(region)
assert "z_mass" in excluded, "z_mass should be excluded from W CR"
assert "z_pt" in excluded, "z_pt should be excluded from W CR"
print(f"✓ {region} correctly excludes z_mass and z_pt\n")

print("=== Z CR Inclusions ===")
region = "1b:CR_Zll_mu"
excluded = plot_manager._get_excluded_variables_for_region(region)
assert "z_mass" not in excluded, "z_mass should NOT be excluded from Z CR"
assert "z_pt" not in excluded, "z_pt should NOT be excluded from Z CR"
print(f"✓ {region} correctly includes z_mass and z_pt\n")

print("All validation tests passed! ✓")


In [ ]:
# Test custom exclusions from config
print("=== Custom Exclusions Test ===")
custom_config = {
    "region_exclusions": {
        "1b:SR": ["custom_var1", "custom_var2"],
        "Top": ["custom_var3"],
    }
}
custom_plot_manager = PlotManager(config=custom_config)

region = "1b:SR"
excluded = custom_plot_manager._get_excluded_variables_for_region(region)
assert "custom_var1" in excluded, "custom_var1 should be excluded"
assert "custom_var2" in excluded, "custom_var2 should be excluded"
print(f"✓ Custom exclusions for {region} work correctly\n")

region = "2b:CR_Top_mu"
excluded = custom_plot_manager._get_excluded_variables_for_region(region)
assert "custom_var3" in excluded, "custom_var3 should be excluded from Top CR"
assert "z_mass" in excluded, "z_mass should still be excluded (default)"
print(f"✓ Custom exclusions for {region} work correctly\n")

print("Custom exclusions validation passed! ✓")
